## this notebook is a test of nucdiff using some fake sequence data

In [49]:
import os, sys, io, random, subprocess
import string
import numpy as np
import pandas as pd
pd.set_option('display.width',600)
from Bio.Seq import Seq, MutableSeq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

from pybioviz import utils

### make two test sequence and make gaps and some mutations in the second

In [60]:
rec = SeqIO.parse('Mbovis.fna',format='fasta').__next__()

s1=rec.seq[:50000] 
seq1 = SeqRecord(s1,id='test1',description='my ref')
#create altered seq
s2 = (s1[:500]+s1[600:]+s1[20000:30000]+s1[35000:40000]).tomutable()
s2[200] = 'T'
s2[1000] = 'A'
seq2 = SeqRecord(s2,id='test2',description='my query')

SeqIO.write(seq1,'test1.fna','fasta')
SeqIO.write(seq2,'test2.fna','fasta')

1

In [61]:
nuc = utils.align_nucmer('test1.fna','test2.fna')

nucmer --maxgap=500 --mincluster=100 --coords -p nucmer test1.fna test2.fna


In [62]:
nuc

,S1,E1,S2,E2,LEN 1,LEN 2,IDENT,TAG1,TAG2
0,1,500,1,500,500,500,99.80,test1,test2
1,601,50000,501,49900,49400,49400,99.99,test1,test2
2,20001,30001,49901,59901,10001,10001,100.00,test1,test2
3,35000,40000,59900,64900,5001,5001,100.00,test1,test2


### run nucdiff and find the differences!

* run nucdiff on the 2 samples with test1 as ref and find deletions plus snps